# Simple investment rule for Ibex 35


With this notebook we have an example of an investment rule with how to download the data from yahoo finance, implement an investment rule, simple but effective, and create two csv, one with today's orders and another one checking the profits. 

The idea of the rule is that you do not have to worry about prices or time, every day at the same time implement the rule.

The next line of research would be to automate the process so that it buys and sells automatically on some page.



In [ ]:
%%time
########################################################################################################################
########################################################################################################################
############################## DOWNLOAD DATA ############################################################
### LOAD LIBRARIES
from pandas_datareader import data
import matplotlib.pyplot as plt
#import pandas as pd
import numpy as np
#import seaborn as sns
from datetime import date
from datetime import datetime
import os
os.chdir('../trading_f/')

pd.set_option('display.max_columns', 500)

## DOWNLOAD DATA
Simbolo  = ["BBVA.MC", "MEL.MC","TEF.MC","ACS.MC","SAN.MC","SAB.MC", "ITX.MC", "ANA.MC", "GRF.MC", "AENA.MC",
 "FER.MC","BKT.MC", "IBE.MC","ACX.MC","MTS.MC", "REE.MC", "AMS.MC", "VIS.MC", "IAG.MC", "SGRE.MC","MAP.MC",
 "COL.MC", "PHM.MC", "CLNX.MC", "ENG.MC",  "ELE.MC", "NTGY.MC","REP.MC", "MRL.MC","CABK.MC", "BKIA.MC", 
 "CIE.MC","IDR.MC","SLR.MC", "ALM.MC"]

Nombre = ["Banco Bilbao Vizcaya Argentaria, S.A.","Meliá Hotels International, S.A.",
"Telefónica, S.A.","ACS, Actividades de Construcción y Servicios, S.A.","Banco Santander, S.A.",
"Banco de Sabadell, S.A.","Industria de Diseño Textil, S.A.","Acciona, S.A.","Grifols, S.A.",
"Aena S.M.E., S.A.","Ferrovial, S.A.","Bankinter, S.A.","Iberdrola, S.A.","Acerinox, S.A.","ArcelorMittal","Red Eléctrica Corporación, S.A.",
"Amadeus IT Group, S.A.","Viscofan, S.A.","International Consolidated Airlines Group, S.A.","Siemens Gamesa Renewable Energy, S.A.",
"Mapfre, S.A.","Inmobiliaria Colonial, SOCIMI, S.A.","Pharma Mar, S.A.","Cellnex Telecom, S.A.","Enagás, S.A.",
"Endesa, Sociedad Anonima","Naturgy Energy Group, S.A.","Repsol, S.A.",
"MERLIN Properties SOCIMI, S.A.","CaixaBank, S.A.","Bankia","CIE Automotive", "Indra","Solaria Energía y Medio Ambiente, S.A.",
"Almirall, S.A"]


EmpresasIbex = pd.DataFrame(
{
    'Empresa' : Nombre,
    'Simbolo' : Simbolo
})

# I WANT THE LAST 3 MONTHS (It may change but I think at least 3 months.)
fecha =  str(date.today().year) +  '-' + str(date.today().month - 3) + '-' + str(date.today().day)

panel_data = data.DataReader(Simbolo, 'yahoo', start= fecha)

# I USE AJD CLOSE, THESE ARE UNREALISTIC  PRICES BUT ARE USEFUL FOR  EXAMPLE
df = pd.DataFrame(panel_data['Adj Close'])


########################################################################################################################
########################################################################################################################
############################## CREATING CALL SERIE ############################################################
df_empty = pd.DataFrame()

for i in Simbolo:
    rent =  np.diff(np.log(df[i]))
    x = i 
    #print(x)
    df_empty[x] = np.where(rent < np.quantile(rent, 0.15),  1, 0)
    #y = i + '_venta'
    #beneficios[y] = np.where(rent > np.quantile(rent, 0.75),  np.diff(data[i]), 0)

# TAKE TODAYS DATA
hoy =  df.index[[-1]].astype(str)[0]

serie_compra = pd.DataFrame(df_empty.iloc[-1, :])
serie_compra.reset_index(inplace = True)
serie_compra.set_axis(['Symbols', 'compra'], axis=1, inplace=True)

# MERGE WITH PRICES
precios = pd.DataFrame(df.iloc[-1, :])
precios.reset_index(inplace= True)
precios.columns = ['Symbols', hoy]

serie_compra = serie_compra.merge(precios, on = 'Symbols', how = 'inner')

# KEEP THE CALL OPTIONS
datos_comprar = serie_compra[serie_compra['compra'] == 1]
datos_comprar.drop(columns = 'compra', inplace = True)
datos_comprar.columns = ['Symbols', hoy]
# UPLOAD CSV, FOR TOMORROWS PROFITS 
datos_comprar.to_csv('../trading_f/lista_compras/' + hoy + '.csv', index= False)

#############################################################################################
##############################################################
#CHECKING YESTERDAY'S BENEFITS
ayer = df.index[[-2]].astype(str)[0]
compras = pd.read_csv('../trading_f/lista_compras/' + ayer + '.csv')

beneficios = compras.merge(serie_compra, how = 'inner', on = 'Symbols')
beneficios['beneficios' + hoy] = beneficios.iloc[:,3] - beneficios.iloc[:,1]
beneficios.to_csv('../trading_f/lista_beneficios/' + hoy + '_beneficios.csv', index= False)
beneficios
